##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
%pip uninstall -qqy jupyterlab kfp
%pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

In [2]:
import os

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [3]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [4]:
DOCUMENT1 = "기후 제어 시스템 작동하기 Googlecar에는 차량의 온도와 공기 흐름을 조절할 수 있는 기후 제어 시스템이 있습니다. 기후 제어 시스템을 작동하려면 중앙 콘솔에 있는 버튼과 손잡이를 사용하세요. 온도: 온도 손잡이는 차량 내부의 온도를 조절합니다. 손잡이를 시계 방향으로 돌리면 온도가 높아지고 시계 반대 방향으로 돌리면 온도가 낮아집니다. 공기 흐름: 공기 흐름 손잡이는 차량 내부의 공기 흐름량을 조절합니다. 손잡이를 시계 방향으로 돌리면 공기 흐름이 높아지고 시계 반대 방향으로 돌리면 공기 흐름이 낮아집니다. 팬 속도: 팬 속도 손잡이는 팬의 속도를 조절합니다. 손잡이를 시계 방향으로 돌리면 팬 속도가 높아지고 시계 반대 방향으로 돌리면 팬 속도가 낮아집니다. 모드: 모드 버튼을 사용하면 원하는 모드를 선택할 수 있습니다. 사용 가능한 모드는 다음과 같습니다. 자동: 차량이 자동으로 온도와 공기 흐름을 조절하여 편안한 수준을 유지합니다. 냉방: 차량이 차량에 차가운 공기를 불어넣습니다. 난방: 차량이 차량에 따뜻한 공기를 불어넣습니다. 제상: 차량이 앞유리에 따뜻한 공기를 불어넣어 성난 눈을 제거합니다."
DOCUMENT2 = 'GoogleCar에는 내비게이션, 엔터테인먼트, 실내 온도 조절 등 다양한 기능을 사용할 수 있는 대형 터치스크린 디스플레이가 있습니다. 터치스크린 디스플레이를 사용하려면 원하는 아이콘을 터치하세요. 예를 들어, "내비게이션" 아이콘을 터치하여 목적지까지 가는 길을 찾거나 "음악" 아이콘을 터치하여 좋아하는 음악을 재생할 수 있습니다.'
DOCUMENT3 = "기어 변속 Googlecar에는 자동 변속기가 장착되어 있습니다. 기어를 변속하려면 변속 레버를 원하는 위치로 옮기기만 하면 됩니다. 주차: 이 위치는 주차 시 사용됩니다. 바퀴가 잠겨 차가 움직일 수 없습니다. 후진: 이 위치는 후진 시 사용됩니다. 중립: 이 위치는 신호등이나 교통 체증에 멈춰 있을 때 사용됩니다. 기어가 들어가 있지 않으므로 가속 페달을 밟지 않으면 차가 움직이지 않습니다. 주행: 이 위치는 전진 시 사용됩니다. 저속: 이 위치는 눈길이나 기타 미끄러운 도로에서 주행할 때 사용됩니다."

# DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
# DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
# DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


In [5]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types

# Define a helper to retry when per-miniute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for docuemnts, or query
    document_mode =True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            # model="models/text-embedding-004",
            model="models/gemini-embedding-exp-03-07",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            )
        )
        return [e.values for e in response.embeddings]

In [6]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [7]:
db.count()

3

In [8]:
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[ 0.02094708,  0.0009292 ,  0.02149961, ..., -0.00051995,
         -0.02071093, -0.00741596]], shape=(1, 3072)),
 'documents': ['기후 제어 시스템 작동하기 Googlecar에는 차량의 온도와 공기 흐름을 조절할 수 있는 기후 제어 시스템이 있습니다. 기후 제어 시스템을 작동하려면 중앙 콘솔에 있는 버튼과 손잡이를 사용하세요. 온도: 온도 손잡이는 차량 내부의 온도를 조절합니다. 손잡이를 시계 방향으로 돌리면 온도가 높아지고 시계 반대 방향으로 돌리면 온도가 낮아집니다. 공기 흐름: 공기 흐름 손잡이는 차량 내부의 공기 흐름량을 조절합니다. 손잡이를 시계 방향으로 돌리면 공기 흐름이 높아지고 시계 반대 방향으로 돌리면 공기 흐름이 낮아집니다. 팬 속도: 팬 속도 손잡이는 팬의 속도를 조절합니다. 손잡이를 시계 방향으로 돌리면 팬 속도가 높아지고 시계 반대 방향으로 돌리면 팬 속도가 낮아집니다. 모드: 모드 버튼을 사용하면 원하는 모드를 선택할 수 있습니다. 사용 가능한 모드는 다음과 같습니다. 자동: 차량이 자동으로 온도와 공기 흐름을 조절하여 편안한 수준을 유지합니다. 냉방: 차량이 차량에 차가운 공기를 불어넣습니다. 난방: 차량이 차량에 따뜻한 공기를 불어넣습니다. 제상: 차량이 앞유리에 따뜻한 공기를 불어넣어 성난 눈을 제거합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.embeddings: 'embeddings'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
# Switch to query mode when generating embeddings
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "터치스크린을 이용해 음악을 재생하려면 어떻게 해야 하나요?"
# query = "How do you use the touchscreen to play music?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])



GoogleCar에는 내비게이션, 엔터테인먼트, 실내 온도 조절 등 다양한 기능을 사용할 수 있는 대형 터치스크린 디스플레이가 있습니다. 터치스크린 디스플레이를 사용하려면 원하는 아이콘을 터치하세요. 예를 들어, "내비게이션" 아이콘을 터치하여 목적지까지 가는 길을 찾거나 "음악" 아이콘을 터치하여 좋아하는 음악을 재생할 수 있습니다.

## 증강 생성: 질문에 답하기
문서 집합에서 관련 구절을 찾았으니(검색 단계), 이제 생성 프롬프트를 구성하여 Gemini API가 최종 답변을 생성하도록 할 수 있습니다. 이 예시에서는 하나의 구절만 검색되었습니다. 실제로, 특히 기반 데이터의 크기가 큰 경우, 두 개 이상의 결과를 검색하고 Gemini 모델이 질문에 대한 답변과 관련된 구절을 판단하도록 하는 것이 좋습니다. 따라서 검색된 구절 중 일부가 질문과 직접적인 관련이 없어도 괜찮습니다. 이 생성 단계에서는 이러한 구절을 무시해야 합니다.

In [11]:
query_online = query.replace("\n", " ")

# 이 프롬프트에서는 어조에 대한 지침이나 모델이 어떤 주제를 고수해야 하는지, 또는 피해야 하는지에 대한 지침을 지정할 수 있습니다.
prompt = f"""아래에 포함된 참고 문헌의 텍스트를 사용하여 질문에 답변하는 유익하고 유용한 봇입니다.
관련된 모든 배경 정보를 포함하여 완전한 문장으로 답변해 주십시오.
하지만 기술적인 지식이 부족한 청중을 대상으로 하고 있으므로 복잡한 개념은 간략하게 설명하고
친근하고 대화하는 듯한 어조를 유지해야 합니다. 답변과 관련 없는 문헌은 무시해도 됩니다.

질문: {query_online}
"""


In [12]:

# Add the retrieved docuemnts to the prompt.
for passage in all_passages:
    passage_online = passage.replace("\n", " ")
    prompt += f"구절: {passage_online}\n"

print(prompt)

아래에 포함된 참고 문헌의 텍스트를 사용하여 질문에 답변하는 유익하고 유용한 봇입니다.
관련된 모든 배경 정보를 포함하여 완전한 문장으로 답변해 주십시오.
하지만 기술적인 지식이 부족한 청중을 대상으로 하고 있으므로 복잡한 개념은 간략하게 설명하고
친근하고 대화하는 듯한 어조를 유지해야 합니다. 답변과 관련 없는 문헌은 무시해도 됩니다.

질문: 터치스크린을 이용해 음악을 재생하려면 어떻게 해야 하나요?
구절: GoogleCar에는 내비게이션, 엔터테인먼트, 실내 온도 조절 등 다양한 기능을 사용할 수 있는 대형 터치스크린 디스플레이가 있습니다. 터치스크린 디스플레이를 사용하려면 원하는 아이콘을 터치하세요. 예를 들어, "내비게이션" 아이콘을 터치하여 목적지까지 가는 길을 찾거나 "음악" 아이콘을 터치하여 좋아하는 음악을 재생할 수 있습니다.



In [13]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt
)

Markdown(answer.text)

터치스크린을 사용하여 음악을 재생하는 것은 매우 쉽습니다! GoogleCar에는 내비게이션, 엔터테인먼트 등 다양한 기능을 제공하는 대형 터치스크린 디스플레이가 장착되어 있습니다. 음악을 재생하려면 "음악" 아이콘을 탭하면 됩니다. 그러면 좋아하는 음악을 들을 수 있습니다.